# Quantum-classical Mapping

## Code for simulating and infering classical Ising dynamics

We want to simulate the dynamics

$W\left(s^{\prime} \mid s\right)=W(s \mid s) \delta_{s^{\prime}, s}+\sum_{K=1,2,\dots k} \sum_{a_k} g_{a_k}(s) \delta_{s^{\prime}, F_{a_k}[s]}$

with $g_a = [{g_i, g_{ij}}]$ the flip-probabilities. The indices refer to which spin will get flipped. w is an extra parameter that allows us to alter the time-scales of certain flips without altering the steady-state distribution.

For fully connected systems where we allow for up to two spin flips we have:

$g_i(s)=\exp \left[-w_{i i}-s_i\left(h_i+\sum_{i \neq j} J_{i j} s_j\right)\right]$

$g_{ij}(s) = \exp \left(-w_{ij}+J_{ij} s_is_j - s_i \left[\sum_{k} J_{ik} s_k + h_i \right] - s_j \left[ \sum_{k'} J_{jk'} s_{k'} + h_j \right] \right)$

Formalism: 
- We map spins {-1,1} to binary {0,1}. 
- The transition matrix W is row-normalized. 
- We set $\beta = 1$ throughout the code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
from scipy.optimize import minimize

In [2]:
###  AUXILARY FUNCTIONS
#----------------------------------------------------------------------
@njit
def index_to_spin_state(s_idx, N):
    """Convert an index to a spin state."""
    s = np.zeros((N,), dtype=np.int64)
    for i in range(N):
        # Get the ith bit of s_idx.
        bit = (s_idx >> i) & 1
        # Convert the bit to a spin (-1 or 1).
        s[N - 1 - i] = bit * 2 - 1
    return s

@njit
def spin_state_to_index(s):
    """Convert a spin state to an index."""
    N = len(s)
    s_idx = 0
    for i in range(N):
        # Convert the spin to a bit (0 or 1).
        bit = (s[i] + 1) // 2
        # Set the ith bit of s_idx.
        s_idx |= bit << (N - 1 - i)
    return s_idx

@njit
def bits_flipped_indices(s_idx, flip_index, N):
    """Return the indices of the bits that are flipped when going from s_idx to flip_index."""
    flipped_bits = s_idx ^ flip_index
    flipped_indices = []
    for k in range(N):
        if (flipped_bits & (1 << k)) != 0:
            flipped_indices.append(N - 1 - k)
    return flipped_indices

@njit
def choice(probabilities):
    """A workaround for np.random.choice, which is unsupported by numba"""
    cumulative_distribution = np.cumsum(probabilities)
    return np.searchsorted(cumulative_distribution, np.random.random(), side ="right")


def flatten_parameters(w, h, J):
    """Flatten the parameter matrices w, h, and J into a single 1D array."""
    return np.hstack((w.ravel(), h.ravel(), J.ravel()))

def unflatten_parameters(params, n_spins):
    """Unflatten the 1D array of parameters back into the matrices w, h, and J."""
    w_size = n_spins**2
    h_size = n_spins

    w = params[:w_size].reshape(n_spins, n_spins)
    h = params[w_size:w_size + h_size]
    J = params[w_size + h_size:].reshape(n_spins, n_spins)

    return w, h, J


In [3]:
###  SIMULATION FUNCTIONS
#----------------------------------------------------------------------
@njit
def g_single(i, s, w, h, J):
    """Probability associated with a single spin flip"""
    sum_term = h[i]
    for j in range(len(s)):
        if i != j:
            sum_term += J[i, j] * s[j]
    return np.exp(-w[i, i] - s[i] * sum_term)


@njit
def g_double(i, j, s, w, h, J):
    """Probability associated with a double spin flip"""
    sum_term_i = h[i]
    for k in range(len(s)):
        if k != i:
            sum_term_i += J[i, k] * s[k]

    sum_term_j = h[j]
    for k in range(len(s)):
        if k != j:
            sum_term_j += J[j, k] * s[k]

    return np.exp(-w[i, j] + J[i, j] * s[i] * s[j] - s[i] * sum_term_i - s[j] * sum_term_j)


@njit
def spin_flip(s, w, h, J, transition_matrix):
    """
    Flipping algorithm using parallel dynamics using double-spin flips.
    Calculates and stores W when needed.
    """
    N = len(s)  # number of spins in the lattice

    # calculate the index of the current state in the transition matrix
    s_idx = spin_state_to_index(s)

    # update the corresponding row in the transition matrix if it's not yet filled
    if not np.any(transition_matrix[s_idx]):

        # calculate all possible flipping probabilities from state s
        flip_probs = np.zeros((2**N))  # Initialize with zeros

        for i in range(N):
            flip_index = s_idx ^ (1 << i)
            i_flipped = bits_flipped_indices(s_idx, flip_index, N)[0]
            flip_probs[flip_index] = g_single(i_flipped, s, w, h, J)  # single flip probabilities

        for i in range(N):
            for j in range(i+1, N):
                flip_index = s_idx ^ (1 << i) ^ (1 << j)
                i_flipped, j_flipped = bits_flipped_indices(s_idx, flip_index, N)
                flip_probs[flip_index] = g_double(i_flipped, j_flipped, s, w, h, J)  # double flip probabilities

        # add the no-flip probability and store the flipping probabilities
        no_flip_prob = 1 - sum(flip_probs)
        flip_probs[s_idx] = no_flip_prob
        transition_matrix[s_idx] = flip_probs
    else:
        flip_probs = transition_matrix[s_idx]   # get probabilities from transition matrix if already calculated

    # randomly choose a spin flip according to its probability of occurring
    idx = choice(flip_probs)

    # apply the chosen spin flip
    flipped_indices = bits_flipped_indices(s_idx, idx, N)
    
    if len(flipped_indices) == 0:  # no flip
        pass
    if len(flipped_indices) == 1:  # single flip
        i = flipped_indices[0]
        s[i] *= -1
    elif len(flipped_indices) == 2:  # double flip
        i, j = flipped_indices
        s[i] *= -1
        s[j] *= -1

    return s, transition_matrix, w

@njit
def find_min_w(w, h, J, n_spins, step_size, no_flip_prob):
    """Finds the smallest offset to add to the w parameter such that diagonal elements of W become at least no_flip_prob"""
    min_scalar = 0
    while True:
        # Update w with the current scalar value
        new_w = w + np.ones(w.shape) * min_scalar
        
        # Calculate the transition matrix
        _, transition_matrix = simulate_and_infer_dynamics(new_w, J, h, steps=1, n_spins=n_spins, fill_missing_entries=True)
        
        # Calculate the flip probabilities for each row in the transition matrix
        flip_probs = np.sum(transition_matrix, axis=1) - np.diag(transition_matrix)

        # Check if the sum of flip probabilities for each row is smaller than (1 - no_flip_prob)
        if np.all(flip_probs < (1 - no_flip_prob)):
            break

        # If the condition is not met, increase the scalar value and try again
        min_scalar += step_size

    return new_w


@njit
def simulate_and_infer_dynamics(w, J, h, steps, n_spins, fill_missing_entries=True):
    """Calculates the transition matrix and a trajectory at the same time"""
    s = np.array([np.random.choice(np.array([-1, 1])) for _ in range(n_spins)], dtype=np.int64) #initial config
    trajectory = np.empty((steps, n_spins), dtype=np.int64)
    # initialize the transition matrix
    transition_matrix = np.zeros((2**n_spins, 2**n_spins))

    for t in range(steps):
        s, transition_matrix, _ = spin_flip(s, w, h, J, transition_matrix)
        trajectory[t] = s

    # If fill_missing_entries is True, fill in the missing rows in the transition matrix after the simulation
    if fill_missing_entries:
        fill_missing_transition_rows(transition_matrix, w, h, J)

    return trajectory, transition_matrix

@njit
def simulate_dynamics(W, steps, n_spins):
    """Only simulate the dynamics of a given transition matrix W"""
    s = np.array([np.random.choice(np.array([-1, 1])) for _ in range(n_spins)], dtype=np.int64) #initial config
    trajectory = np.empty((steps, n_spins), dtype=np.int64)

    for t in range(steps):
        s_idx = spin_state_to_index(s)
        flip_probs = W[s_idx]
        idx = choice(flip_probs)

        if idx == s_idx:  # no flip
            pass
        elif idx in [s_idx ^ (1 << i) for i in range(n_spins)]:  # single flip
            i = bits_flipped_indices(s_idx, idx, n_spins)[0]
            s[i] *= -1
        else:  # double flip
            i, j = bits_flipped_indices(s_idx, idx, n_spins)
            s[i] *= -1
            s[j] *= -1

        trajectory[t] = s

    return trajectory

@njit
def fill_missing_transition_rows(transition_matrix, w, h, J):
    """Fill in the missing rows in the transition matrix after the simulation"""
    n_spins = int(np.log2(transition_matrix.shape[0]))
    for s_idx in range(transition_matrix.shape[0]):
        if np.any(transition_matrix[s_idx]):
            continue
        # convert the index to a spin state
        s = index_to_spin_state(s_idx, n_spins)
        spin_flip(s, w, h, J, transition_matrix)

In [4]:
###  PLOTTING FUNCTIONS
#----------------------------------------------------------------------

def plot_combined_dynamics(trajectory, n_spins):
    fig, (ax1,ax2)= plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot for the system's evolution through states
    states = [(''.join(['0' if spin == -1 else '1' for spin in s])) for s in trajectory]  # convert spins to binary
    unique_states = sorted(set(states))
    state_indices = [unique_states.index(state) for state in states]
    
    ax1.step(range(len(state_indices)), state_indices)
    ax1.set_yticks(range(len(unique_states)))
    ax1.set_yticklabels(unique_states)
    ax1.set_title("System's evolution through states")
    ax1.set_xlabel('Time step')
    ax1.set_ylabel('State')

    # Plot for the spins' evolution
    offset_labels = []
    y_ticks = []
    for i in range(n_spins):
        spin_trajectory = trajectory[:, i] + i * 3
        ax2.step(range(len(spin_trajectory)), spin_trajectory, label=f'Spin {i+1}')
        offset_labels += ['0', '1']
        y_ticks += [(i*3 - 1), (i*3 +1)]
        
    ax2.set_title("Spins' evolution")
    ax2.set_xlabel('Time step')
    ax2.set_ylabel('Spin value (with offset)')
    ax2.set_yticks(y_ticks)
    ax2.set_yticklabels(offset_labels)
    ax2.legend(loc='best')
    ax2.plot()
    
    plt.tight_layout()
    plt.show()

def plot_scatter(ax, x, y, xlabel, ylabel, color, size=10):
    '''Creates a scatter subplot'''
    ax.scatter(x, y, s=size, marker='o', color=color)
    ax.set_xlabel(xlabel, fontsize=20)
    ax.set_ylabel(ylabel, fontsize=20)
    ax.set_yscale('log')

def plot(it, KL, Wmax, title='Convergence Plots', size=10):
    fig = plt.figure(figsize=(25, 6));                           #  make plots
    fig.suptitle(title, fontsize=30, y = 1)
    trans = mtransforms.ScaledTranslation(-20/72, 7/72, fig.dpi_scale_trans)
    its= np.arange(1,it+1,1)

    ax1 = fig.add_subplot(1, 2, 1)
    plot_scatter(ax1, its, Wmax[:it], "Iterations", r"$ (\Delta w)_{max}$", "Coral", size)
    ax1.text(0, 1.0, 'A.)', transform=ax1.transAxes + trans, fontsize='large',fontweight ='bold', va='bottom', fontfamily='sans-serif')
   
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.scatter(its, KL[:it], s=size, marker='o', color="ForestGreen")
    ax2.set_xlabel("Iterations", fontsize=20)
    ax2.set_ylabel("KL Divergence", fontsize=20)
    ax2.text(0, 1.0, 'B.)', transform=ax2.transAxes + trans, fontsize='large',fontweight ='bold', va='bottom', fontfamily='sans-serif')


In [5]:
###  INFERENCE FUNCTIONS
#----------------------------------------------------------------------
@njit
def infer_transition_matrix(trajectory):
    """Infers the transition matrix from a time series of the states of the system"""
    n_spins = trajectory.shape[1]
    n_states = 2**n_spins

    # initialize the transition matrix
    transition_matrix = np.zeros((n_states, n_states))

    # count the transitions from each state to each other state
    for i in range(len(trajectory) - 1):
        # Convert the spin states to indices
        from_idx = spin_state_to_index(trajectory[i])
        to_idx = spin_state_to_index(trajectory[i+1])

        # Update the transition matrix
        transition_matrix[from_idx, to_idx] += 1

    # Normalize each row of the transition matrix
    for i in range(n_states):
        row_sum = transition_matrix[i].sum()
        if row_sum > 0:
            transition_matrix[i] /= row_sum

    return transition_matrix


def negative_log_likelihood(params, trajectory, n_spins):
    """Compute the negative log-likelihood of the observed trajectory given the model parameters."""

    # Unflatten the parameters
    w, h, J = unflatten_parameters(params, n_spins)

    # Calculate the transition matrix with the current values of w, h, and J
    _, transition_matrix = simulate_and_infer_dynamics(w, J, h, steps=1, n_spins=n_spins, fill_missing_entries=True)

    # Compute the negative log-likelihood
    nll = 0
    for i in range(trajectory.shape[0] - 1):
        s = trajectory[i]
        s_next = trajectory[i + 1]
        s_idx = spin_state_to_index(s)
        s_next_idx = spin_state_to_index(s_next)

        # Probability of transitioning from s to s_next
        prob = transition_matrix[s_idx, s_next_idx]

        if prob > 0:
            nll -= np.log(prob)

    return nll

def infer_parameters(trajectory, n_spins, initial_params=None):
    """Infer the parameter matrices w, h, and J by minimizing the likelihood from a time series data of the states the system went through."""

    if initial_params is None:
        # Create initial guess for the parameters
        w_init = np.random.rand(n_spins, n_spins)
        h_init = np.random.rand(n_spins)
        J_init = np.random.rand(n_spins, n_spins)

        initial_params = flatten_parameters(w_init, h_init, J_init)

    # Minimize the negative log-likelihood using the BFGS algorithm
    result = minimize(negative_log_likelihood, initial_params, args=(trajectory,n_spins), method='BFGS')

    # Unflatten the optimized parameters back into matrices
    w_opt, h_opt, J_opt = unflatten_parameters(result.x, n_spins)

    return w_opt, h_opt, J_opt

def boltzmann_distribution(J, h):
    N = len(h)
    all_states = np.array([index_to_spin_state(s_idx, N) for s_idx in range(2**N)])
    energies = np.array([energy(state, J, h) for state in all_states])
    probabilities = np.exp(-energies)
    Z = np.sum(probabilities)
    return probabilities / Z


def energy(s, J, h):
    """
    Compute the energy of the system in a particular spin state.
    """
    s = s.astype(np.float64)                              # convert the spin configuratio to float64 for numba
    pairwise_energy = np.dot(s, np.dot(J, s))
    local_field_energy = h * s                            # simple multiplication for scalar-array operation
    return -pairwise_energy - np.sum(local_field_energy)  # sum over local field energy


In [6]:
###  2-QUBIT QBM FUNCTIONS
#----------------------------------------------------------------------
@njit
def expmat(A):
    """
    Computes the exponential of a given matrix `A'.
    """
    A = 0.5 * (A + np.transpose(np.conjugate(A)))
    evals, evecs = np.linalg.eigh(A)
    N = len(evals)
    res = np.zeros((N,N),np.complex64)
    for i in range(N):
        eigenvector = evecs[:,i]
        projector = np.outer(eigenvector,eigenvector.conj())
        res += np.exp(evals[i]) * projector
    return res

@njit
def logmat(A):
    """
    Computes the natural logarithm of a given matrix `A`.
    """
    A = 0.5 * (A + np.transpose(np.conjugate(A)))
    evals, evecs = np.linalg.eigh(A)
    N = len(evals)
    res = np.zeros((N,N),np.complex64)
    for i in range(N):
        eigenvector = evecs[:,i]
        projector = np.outer(eigenvector,eigenvector)
        res += np.log(evals[i]) * projector
    return res

def generate_interaction_matrices():
    """
    Constructs the tensor product of Pauli matrices for each spin and 
    uses them to build the interaction matrices between the two spins.
    """
    pauI = np.array([[1,0],[0,1]],np.complex64)
    pauX = np.array([[0,1],[1,0]],np.complex64)
    pauY = np.array([[0,-1j],[1j,0]],np.complex64)
    pauZ = np.array([[1,0],[0,-1]],np.complex64)
    pau = np.array([pauI, pauX, pauY, pauZ], dtype=np.complex64)

    interactions = np.zeros((4, 4, 4, 4), dtype=np.complex64)
    for k in range(4):
        for kprime in range(4):
            interactions[k, kprime] = np.kron(pau[k], pau[kprime])
    return interactions

@njit
def hamiltonian2spins(J,interactions):
    """
    Calculates the Hamiltonian matrix of a two-spin system 
    """
    H = np.zeros((4,4),np.complex64)
    for k in range(4):
        for kprime in range(4):
            H += J[k,kprime] * interactions[k,kprime]
    return H

@njit
def rho_model(w,interactions):
    """
    Computes the density matrix of a two-spin system using the Hamiltonian
    and the interaction matrices, and then normalizes it. Exact Diagonalization.
    """
    H      = hamiltonian2spins(w,interactions)        #  get hamiltonian matrix
    rho    = expmat(H)                                #  defintion of rho
    Z      = np.real(np.trace(rho))                   #  get Z
    rho   /= Z                                        #  normalize such that Tr[rho] = 1
    return rho

@njit
def observables(rho, interactions):
    """
    Computes the expectation values of the interaction matrices [observables]
    """
    obs = np.zeros((4,4))
    for k in range(4):
        for kprime in range(4):
            obs[k,kprime] = np.real(np.trace(rho@interactions[k,kprime]))
    return obs

@njit
def KL_divergence(eta,rho):
    """
    Calculates the KL divergence between the model and target distribution.
    """
    return np.real(np.trace(eta@(logmat(eta)-logmat(rho))))

@njit
def generate_random_parameter_matrix(seed):
    np.random.seed(seed)              # set random seed for reproducibility
    w = np.random.randn(4, 4)         # get normal distributed weights
    w[0, 0] = 0
    return w

def generate_w(wx1x2, wy1y2, w1z2z, hx1, hx2, hy1, hy2, hz1, hz2):
    return np.array([
        [0,  hx2, hy2, hz2],
        [hx1, wx1x2, 0,  0],
        [hy1, 0,  wy1y2, 0],
        [hz1, 0,  0,  w1z2z]
    ])

@njit
def quantum_boltzmann_machine(interactions, lr, maxiter, tol, random_seed, w_eta=None, eta=None):
    """
    Train the model to fit the target distribution eta
    """
    if w_eta is not None:
        eta = rho_model(w_eta, interactions)               # compute density matrix using ED if w_eta is provide
    obs_clamped  = observables(eta, interactions)          # get QM clamped statistics  
    
    w    = generate_random_parameter_matrix(random_seed)  # initialize random parameters
    rho  = rho_model(w, interactions)                      # compute density matrix using ED
    obs  = observables(rho, interactions)                  # get QM statistics  
    
    it = 0                                                 #  initialize gradient ascent loop
    diff = np.max(np.abs(obs-obs_clamped))
    Wmax_list = np.zeros(maxiter)                          #  initialize values to store
    KL_list   = np.zeros(maxiter)
    
    
    while (diff > tol and it < maxiter): 
        rho  = rho_model(w, interactions)                   # compute density matrix using ED
        obs  = observables(rho, interactions)               # get QM statistics  
        w              += lr * (obs_clamped -  obs)         #  update parameters
        
        diff = np.max(np.abs(obs - obs_clamped))            #  evaluate differences in clamped and model statistics   
        if w_eta is not None:
            Wmax = np.max(np.abs(w-w_eta))
            Wmax_list[it] = Wmax                                #  store values
        
        KL_list[it]   = KL_divergence(eta,rho) 
        it += 1 
    return w, rho, KL_list, Wmax_list, it

In [14]:
### QBM <-> ISING DYNAMICS FUNCTIONS
#----------------------------------------------------------------------
# @njit
def transition_matrix_to_density_matrix(W):
    # calculate steady-state distribution p
    eigenvalues, eigenvectors = np.linalg.eig(W.T) #switch row to collumn normalized by transposing

    p = np.abs(eigenvectors[:, np.argmax(np.real(eigenvalues))])
    p = p / np.sum(p) #normalize 

    # calculate the matrix A
    sqrt_p_diag     = np.diag(np.sqrt(p))
    inv_sqrt_p_diag = np.diag(1 / np.sqrt(p))
    A = inv_sqrt_p_diag @ W.T @ sqrt_p_diag

    # symmetrize A to make it Hermitian
    A = 0.5 * (A + np.transpose(np.conjugate(A)))

    # add a scalar constant to all eigenvalues to make them non-negative (ensuring rho is positive semidefinite)
    min_eigval = np.min(np.real(np.linalg.eigvals(A)))
    if min_eigval < 0:
        A += (np.abs(min_eigval) + 1e-4) * np.eye(A.shape[0])

    #  normalize such that Tr[rho] = 1
    rho = A / np.real(np.trace(A))    
    rho = rho.astype(np.complex64) #make it complex    

    return rho

# @njit
def density_matrix_to_transition_matrix(rho):
    # obtain the steady-state distribution √p
    eigenvalues, eigenvectors = np.linalg.eigh(rho)
    sqrt_p = np.abs(eigenvectors[:, np.argmax(np.real(eigenvalues))])

    # square the elements of the eigenvector and normalize to obtain the steady-state distribution
    p = sqrt_p**2
    p = p / np.sum(p)  # normalize

    sqrt_p_diag     = np.diag(np.sqrt(p))
    inv_sqrt_p_diag = np.diag(1 / np.sqrt(p))

    # calculate the matrix A
    A = np.real(rho)

    # compute the transition matrix W 
    W = sqrt_p_diag @ A @ inv_sqrt_p_diag
    W = W.T #switch back to row normalized by transposing

    # normalize each row to ensure the sum of each row equals 1
    W_row_sum = W.sum(axis=1, keepdims=True)
    W /= W_row_sum
    return W

def check_ergodicity(rho, K=10):
    """
    Check if the density matrix rho is ergodic by iteratively squaring
    the matrix up to K times and checking for non-zero entries.
    """
    for _ in range(K):
        if np.all(rho > 0):
            return True
        
        # quadratic of the density matrix
        rho = rho @ rho

    # if after K iterations, there are still zero components, the matrix is not ergodic.
    return False

def check_single_flip_ergodicity(rho, W, n_spins):
    """
    Check if the components of the density matrix rho that correspond to 
    the transitions of the transition matrix W involving single spin flips are non-zero.
    """
    n_states = 2 ** n_spins

    for from_idx in range(n_states):
        for i in range(n_spins):
            # calculate the index of the state with the ith spin flipped
            to_idx = from_idx ^ (1 << i)

            # if the transition corresponds to a single spin flip and the component in rho is zero, return False
            if W[from_idx, to_idx] > 0 and rho[from_idx, to_idx] == 0:
                return False

    # if all components corresponding to single spin flips are non-zero, return True
    return True

## PPT slide recreation

- Homogenous systems first [all the same]
- biases <-> sigma z
- Make a way to systematically review the change in parameters. Plots of the change in parameters versus the change in an other

infer_parameters()  [Combo of BM ]
forward_mapping()
inverse_mapping()
forward_mapping_plot
inverse_mapping_plot


Forward mapping

In [15]:
n_spins = 2
w = np.array([[12.,12.],
              [12.,12.]])
J = np.array([[0,0],
              [0,0,]])
h = np.array([0,  0])


steps = 1500

# #min w parameters
# step_size = 0.01
# min_no_flip_prob = 0           #
# min_w = find_min_w(w, h, J, n_spins, step_size, min_no_flip_prob)

#QBM parameters
lr = 0.6
maxiter = 2**20
tol = 1e-6
random_seed = 777

#GET W 
trajectory, W = simulate_and_infer_dynamics(w,J,h,steps,n_spins)

#W TO eta
eta = transition_matrix_to_density_matrix(W)

#RHO TO QM HAMILTONIAN
interactions     = generate_interaction_matrices()    
w_qm, _, _, _, _ = quantum_boltzmann_machine(interactions, lr, maxiter, tol, random_seed, eta = eta)
np.set_printoptions(precision=8, suppress=True)
print(f"Classical parameters: \n w: \n {w} \nJ: \n {J} \n h: \n {h}")
print(f"QM parameters: \n {w_qm}")

0.9999754231505864
Classical parameters: 
 w: 
 [[12. 12.]
 [12. 12.]] 
J: 
 [[0 0]
 [0 0]] 
 h: 
 [0 0]
QM parameters: 
 [[-0.          0.00000603 -0.00000011  0.00000008]
 [ 0.00000626  0.00000635  0.         -0.00000015]
 [-0.00000006  0.0000001  -0.00000011  0.00000018]
 [-0.00000011 -0.0000001  -0.00000022 -0.00000005]]


In [11]:
n_spins = 2
w = np.array([[1.2,2.0],
              [2.0,1.2]])
J = np.array([[0,2],
              [2,0,]])
h = np.array([0,  0])


steps = 1500

# #min w parameters
# step_size = 0.01
# min_no_flip_prob = 0           #
# min_w = find_min_w(w, h, J, n_spins, step_size, min_no_flip_prob)

#QBM parameters
lr = 0.2
maxiter = 2**20
tol = 1e-6
random_seed = 777

#GET W 
trajectory, W = simulate_and_infer_dynamics(w,J,h,steps,n_spins)

#W TO eta
eta = transition_matrix_to_density_matrix(W)

#RHO TO QM HAMILTONIAN
interactions     = generate_interaction_matrices()    
w_qm, _, _, _, _ = quantum_boltzmann_machine(interactions, lr, maxiter, tol, random_seed, eta = eta)
np.set_printoptions(precision=2, suppress=True)
print(f"Classical parameters: \n w: \n {w} \nJ: \n {J} \n h: \n {h}")
print(f"QM parameters: \n {w_qm}")

Classical parameters: 
 w: 
 [[1.2 2. ]
 [2.  1.2]] 
J: 
 [[0 2]
 [2 0]] 
 h: 
 [0 0]
QM parameters: 
 [[ 0.    0.08  0.   -0.  ]
 [ 0.08  2.44 -0.   -0.  ]
 [-0.    0.    2.45  0.  ]
 [ 0.    0.   -0.    3.03]]


Inverse route

In [10]:
# QM HAMILTONIAN PARAMETERS
wx1x2 = 2
wy1y2 = 0
wz1z2 = 0
hx1   = 1
hx2   = 1
hy1   = 0
hy2   = 0
hz1   = 0
hz2   = 0

w_eta            = generate_w(wx1x2 ,wy1y2, wz1z2, hx1, hx2, hy1, hy2, hz1, hz2)   

#QUANTUM HAMILTONIAN TO RHO
interactions     = generate_interaction_matrices()     
_, rho, _, _, _ = quantum_boltzmann_machine(interactions, lr, maxiter, tol, random_seed, w_eta = w_eta)

#RHO TO W
W = density_matrix_to_transition_matrix(rho)
if not check_ergodicity(rho, K=10):
    print('error')

np.set_printoptions(precision=2, suppress=True)
# SIMULATE SYSTEM & INFER PARAMETERS
trajectory = simulate_dynamics(W, steps, n_spins)
w_opt, h_opt, J_opt = infer_parameters(trajectory, n_spins, initial_params=None)

print(f"Input QM parameters: \n {w_eta}")
print(f"Inferred w: \n {w_opt}")
print(f"Inferred h: \n {h_opt}")
print(f"Inferred J: \n {J_opt}")

/Users/ringo/opt/anaconda3/envs/QBM/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Users/ringo/opt/anaconda3/envs/QBM/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Input QM parameters: 
 [[0 1 0 0]
 [1 2 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Inferred w: 
 [[-0.26  0.56]
 [-0.48 -0.47]]
Inferred h: 
 [0.91 0.53]
Inferred J: 
 [[0.14 0.46]
 [1.04 0.15]]


/Users/ringo/opt/anaconda3/envs/QBM/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
